In [ ]:
import os
import glob
import json
import numpy as np
import pandas as pd
import polars as pl
from dotenv import load_dotenv

In [4]:
load_dotenv(r"/rdrive/workspace/perfectdays/.env")

True

In [5]:
NEWS_PARQUET_MONTH_DIR = os.environ["NEWS_PARQUET_MONTH_DIR"]

In [6]:
mmfiles = sorted(glob.glob(os.path.join(NEWS_PARQUET_MONTH_DIR, "*3pty_ko.npz")))
mmfiles[:5]

['/rdrive/rtrs_news/monthly/1999-03_embeddings_3pty_ko.npz',
 '/rdrive/rtrs_news/monthly/1999-04_embeddings_3pty_ko.npz',
 '/rdrive/rtrs_news/monthly/1999-05_embeddings_3pty_ko.npz',
 '/rdrive/rtrs_news/monthly/1999-06_embeddings_3pty_ko.npz',
 '/rdrive/rtrs_news/monthly/1999-07_embeddings_3pty_ko.npz']

In [7]:
mmfile = mmfiles[-1]

In [6]:
# read npz file
mm_data = np.load(mmfile)

In [7]:
mm_data.keys()

KeysView(NpzFile 'L:\\MED\\TRAN\\2025_BankRegDataCollection\\rtrs_news\\monthly\\2025-10_embeddings_3pty_ko.npz' with keys: ids, embeddings)

In [8]:
# print shape of each array
for key in mm_data.keys():
    print(f"{key}: {mm_data[key].shape}")   

ids: (173580,)
embeddings: (173580, 768)


In [13]:
# merge id and embedding into a dataframe
mm_df = pl.DataFrame({
    "id": mm_data["ids"],
    "embedding": [emb for emb in mm_data["embeddings"]]
})

In [29]:
mm_df

id,embedding
i64,"array[f32, 768]"
0,"[0.00873, -0.014797, … -0.004965]"
1,"[0.014095, -0.009662, … 0.000396]"
2,"[0.021123, 0.010042, … 0.02011]"
3,"[0.01345, -0.01291, … 0.005197]"
4,"[0.005833, -0.001751, … -0.020308]"
…,…
173575,"[0.002113, -0.013208, … -0.000254]"
173576,"[0.031447, -0.00351, … 0.006743]"
173577,"[0.031447, -0.00351, … 0.006743]"


In [30]:
mm_df.shape

(173580, 2)

In [8]:
newsfile = os.path.join(NEWS_PARQUET_MONTH_DIR, '2025-10.parquet')

In [10]:
dfnews = pd.read_parquet(newsfile)
dfnews['ids'] = dfnews.index.astype(str)
dfnews.head(2)

,guid,version_created,title,lang_code,subject_qcodes,content,src,ids
0,"tag:reuters.com,2025-10-01:newsml_BwHJtdMa:1",2025-10-01T00:00:00.078Z,VFC INVESTORS: Kirby McInerney LLP Reminds V.F...,en,"B:1221, B:1323, B:195, B:234, B:239, B:242, B:...",For best results when printing this announceme...,3PTY,0
1,"tag:reuters.com,2025-10-01:newsml_Bw2HLJs9a:1",2025-10-01T00:00:00.116Z,Thermo Fisher Scientific Prices Offering of US...,en,"A:2, B:148, B:149, B:150, B:151, B:156, B:1700...",For best results when printing this announceme...,3PTY,1


In [11]:
# sampling 1% of dfnews
dfnews = dfnews[(dfnews['lang_code'] == 'ko') & (dfnews['src'] == '3PTY')]
# 1% sampling
dfnews = dfnews.sample(frac=0.01, random_state=42)
dfnews.shape

(1736, 8)

In [12]:
dfnews['text'] = dfnews.apply(lambda x: x['title'] + '\n\n' + x['content'], axis=1)
dfnews.head(2)

,guid,version_created,title,lang_code,subject_qcodes,content,src,ids,text
1785758,"tag:reuters.com,2025-10-16:newsml_EDYG00389:1",2025-10-16T22:48:44.801Z,"[AI시그널] 티에프이, 상승세 지속 중 거래량 증가로 긍정적 신호",ko,"B:163, B:164, B:1740, B:278, B:279, G:1, G:6, ...",\r\n\r\n\r\n\r\n\r\n [이데일리 이머니 기자] 기술적 ...,3PTY,1785758,"[AI시그널] 티에프이, 상승세 지속 중 거래량 증가로 긍정적 신호\n\n\r\n\..."
2013268,"tag:reuters.com,2025-10-18:newsml_EDYI00023:2",2025-10-18T23:23:55.309Z,고1 ‘성적 인플레’ 심화…내신 경쟁 ‘원점수’까지 확대,ko,"M:1QD, M:2CX",- 내신 9등급→ 5등급 개편되며 변별력 약화\r\n- 학생 간 내신 경쟁 ‘등급’...,3PTY,2013268,고1 ‘성적 인플레’ 심화…내신 경쟁 ‘원점수’까지 확대\n\n- 내신 9등급→ 5...


In [13]:
from transformers import pipeline
import torch

model_id = r'/rdrive_pvc/huggingface_cache/hub/models--unsloth--gpt-oss-120b/snapshots/212ea6d47b66c92f5bfd27956ef07bc160f5ea68'

pipe = pipeline(
    "text-generation",
    model=model_id,
    dtype="auto",
    device_map="auto",
)

messages = [
    {"role": "user", "content": "Explain quantum mechanics clearly and concisely."},
]

outputs = pipe(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])


/rdrive_pvc/Env/openaioss/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 15/15 [00:42<00:00,  2.81s/it]
Device set to use cuda:0


{'role': 'assistant', 'content': "analysisWe need to produce a clear, concise explanation of quantum mechanics, likely for a general audience, maybe with some technical depth but not overwhelming. Should cover core principles: wave-particle duality, superposition, quantization, uncertainty principle, measurement, entanglement, Schrödinger equation, etc. Use analogies. Keep concise but thorough. Provide bullet points. Probably also note differences from classical physics. Provide a short summary at end. We'll output in a friendly tone.assistantfinal**Quantum Mechanics in a Nutshell**\n\n---\n\n### 1. What it is  \nQuantum mechanics (QM) is the physical theory that describes how **tiny systems**—atoms, electrons, photons, and other elementary particles—behave.  \nAt these scales the world stops looking like the everyday “classical” world and follows a different set of rules.\n\n---\n\n### 2. Core Principles  \n\n| Principle | What it means | Simple analogy |\n|-----------|---------------

## Financial Sentiment Prompt

In [ ]:
sentiment_prompt = (
    "You are a financial market analyst. Analyse the Korean news excerpt below and "
    "determine the sentiment toward affected financial assets. Follow these rules:"
    "\n1. Sentiment must be one of POSITIVE, NEGATIVE, NEUTRAL."
    "\n2. Base the assessment on expected market impact (equity, credit, FX, macro risk)."
    "\n3. Provide a one-sentence justification referencing concrete drivers."
    "\n4. Respond strictly in JSON with keys 'sentiment' and 'justification'."
    "\n5. Use the exact structure shown in the example."
    "\n\nExample response:"
    "\n{\n  \"sentiment\": \"NEGATIVE\",\n  \"justification\": \"Bond yields rise as the report signals higher default risk for Korean banks.\"\n}"
    "\n\nText: {text}"
 )

In [ ]:
def analyse_sentiment(text: str) -> dict[str, str]:
    messages = [
        {"role": "user", "content": sentiment_prompt.format(text=text)},
    ]
    outputs = pipe(messages, max_new_tokens=128)
    content = outputs[0]["generated_text"][-1]["content"].strip()
    try:
        parsed = json.loads(content)
        return {
            "sentiment": parsed.get("sentiment", "UNKNOWN"),
            "justification": parsed.get("justification", ""),
        }
    except json.JSONDecodeError:
        return {
            "sentiment": "UNKNOWN",
            "justification": content,
        }

In [ ]:
sample_indices = dfnews.sample(n=min(3, 5), random_state=0).index

sentiment_samples = []
for idx in sample_indices:
    result = analyse_sentiment(dfnews.loc[idx, "text"])
    sentiment_samples.append({
        "index": idx,
        "headline": dfnews.loc[idx, "title"],
        "sentiment": result["sentiment"],
        "justification": result["justification"],
    })

,index,headline,sentiment
0,1218843,"'도시문제' 해결책 찾아…충남혁신센터-천안시, 스타트업 발굴",analysisWe need to classify sentiment toward i...
1,1462049,"AIBIZ, 세미콘웨스트 3년 연속 참가.. '글로벌 반도체 협력 모색'",analysisWe need to classify sentiment toward i...
2,1276109,"'어린이 위협 시구' 논란 최현욱, 자필 편지로 사과 전할까",analysisWe need to classify sentiment toward i...


In [17]:
sentiment_samples[0]

{'index': 1218843,
 'headline': "'도시문제' 해결책 찾아…충남혁신센터-천안시, 스타트업 발굴",
 'sentiment': 'analysisWe need to classify sentiment toward impacted financial assets as POSITIVE, NEGATIVE, or NEUTRAL. The news is about a partnership between Chungnam Innovation Center and Cheonan City Corp to discover/startup in smart city, open innovation, providing testing, investment linkage. This is generally positive for startups, local public sector, maybe equity of related firms, but not immediate macro impact. Likely neutral to positive. Since it mentions investment linking, support, could boost equities of related tech firms and local government bonds maybe positive. So label: POSITIVE. One sentence justification.assistantfinal**POSITIVE**'}

In [ ]:
dfres = pd.DataFrame(sentiment_samples)
dfres

In [ ]:
sentiment_samples